In [1]:
import torch, humanize, mmcv, mmdet
print("PyTorch version:", torch.__version__)
print("GPU available:", torch.cuda.is_available())
print("GPU device count:", torch.cuda.device_count())
print("CUDA version:", torch.version.cuda)
print(f'CUDA max memory allocated: {humanize.naturalsize(torch.cuda.max_memory_allocated())}')
print("PyTorch CUDA archs:", torch.cuda.get_arch_list())
print("mmcv version:", mmcv.__version__)
print("mmdet version:", mmdet.__version__) #

/home/ck37/.conda/envs/mmdet/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PyTorch version: 1.13.1
GPU available: True
GPU device count: 4
CUDA version: 11.7
CUDA max memory allocated: 0 Bytes
PyTorch CUDA archs: ['sm_37', 'sm_50', 'sm_60', 'sm_61', 'sm_70', 'sm_75', 'sm_80', 'sm_86', 'compute_37']
mmcv version: 1.6.0
mmdet version: 2.28.2


In [2]:
from pathlib import Path
import pyprojroot
dir_proj = pyprojroot.here()
print("Project directory:", dir_proj)
dir_mm = dir_proj / "external/mmdetection"
print(dir_mm)

Project directory: /home/ck37/projects/ecig-vaping
/home/ck37/projects/ecig-vaping/external/mmdetection


## Model setup

In [3]:
# Copied from mmdet-train.ipynb and video-evaluation.ipynb
model_name = "vfnet_x101_64x4d_fpn_mdconv_c3-c5_mstrain_2x_coco"
timestr = "20220523-105448"

model_name = "dyhead" # 2023-04-17
timestr = "20230325-193432" # 2023-03-25

dir_models = Path("../models/")

path_model = dir_models / str(model_name + ".py")
print("Model file:", path_model)
print("Find model:", path_model.is_file())

file_checkpoint = dir_proj / ('notebooks/work_dirs/' + model_name + "-" + timestr +'/latest.pth')
print("Checkpoint file:", file_checkpoint)
print("Find checkpoint:", file_checkpoint.is_file())

Model file: ../models/dyhead.py
Find model: True
Checkpoint file: /home/ck37/projects/ecig-vaping/notebooks/work_dirs/dyhead-20230325-193432/latest.pth
Find checkpoint: True


## Test model

In [9]:
import time
timestr = time.strftime("%Y%m%d-%H%M%S")

!{dir_mm}/tools/dist_test.sh {path_model} \
  {file_checkpoint} \
  2 \
  --eval bbox \
  --options "classwise=True" \
  --work-dir work_dirs/test-{model_name}-{timestr} \
  --out work_dirs/test-{model_name}-{timestr}/out.pkl

/home/ck37/.conda/envs/mmdet/lib/python3.9/site-packages/torch/distributed/launch.py:180: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
/home/ck37/projects/ecig-vaping/external/mmdetection/tools/test.py:116: UserWarning: --options is deprecated in favor of --eval-options
  warnings.warn('--options is deprecated in favor of --eval-options')
/home/ck37/p

In [10]:
!ls work_dirs/test-{model_name}-{timestr}

eval_20230417_224913.json


In [11]:
!ls

analyze-unlabeled-videos.Rmd	ocr.ipynb
compile-video-results.ipynb	ocr-nlp.ipynb
dataset_analysis		out.pkl
detect-faces.ipynb		score-videos.ipynb
detect-faces.py			v7-annotations-to-coco.ipynb
face-recognition-testing.ipynb	video_demo-ck.py
mmclass-train.ipynb		video-evaluation.ipynb
mmdet-test.ipynb		work_dirs
mmdet-train.ipynb


## Confusion matrix plot

In [19]:
#work_dirs/test-{model_name}-{timestr}/out.pkl \
!python {dir_mm}/tools/analysis_tools/confusion_matrix.py \
    {path_model} \
    out.pkl \
    work_dirs/test-{model_name}-{timestr}

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 860/860, 2884.5 task/s, elapsed: 0s, ETA:     0s

In [20]:
!ls work_dirs/test-{model_name}-{timestr}

confusion_matrix.png  eval_20230417_224913.json
